I got this dataset from kaggle
You can get the dataset from kaggle using this link : https://www.kaggle.com/datasets/gmudit/employer-data


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pyttsx3 as pt

In [2]:
data_frame=pd.read_csv("Employers_data.csv")
print(data_frame.shape)
print(data_frame.info())
print(data_frame.describe())

(10000, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Employee_ID       10000 non-null  int64 
 1   Name              10000 non-null  object
 2   Age               10000 non-null  int64 
 3   Gender            10000 non-null  object
 4   Department        10000 non-null  object
 5   Job_Title         10000 non-null  object
 6   Experience_Years  10000 non-null  int64 
 7   Education_Level   10000 non-null  object
 8   Location          10000 non-null  object
 9   Salary            10000 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 781.4+ KB
None
       Employee_ID           Age  Experience_Years         Salary
count  10000.00000  10000.000000      10000.000000   10000.000000
mean    5000.50000     35.455900         12.370900  115381.500000
std     2886.89568     10.000213          9.148951   46066.139047
min    

In [3]:
engine=pt.init()
voices=engine.getProperty('voices')
engine.setProperty('voice',voices[2].id)
engine.say(data_frame.shape)
engine.runAndWait()
del engine

In [4]:
print(data_frame.isnull().sum())
print(data_frame.duplicated().sum())

Employee_ID         0
Name                0
Age                 0
Gender              0
Department          0
Job_Title           0
Experience_Years    0
Education_Level     0
Location            0
Salary              0
dtype: int64
0


In [5]:
print(data_frame.dtypes)

Employee_ID          int64
Name                object
Age                  int64
Gender              object
Department          object
Job_Title           object
Experience_Years     int64
Education_Level     object
Location            object
Salary               int64
dtype: object


In [6]:
print("we do not need the employee ID and the name ")
print(data_frame.sample(2))

we do not need the employee ID and the name 
      Employee_ID              Name  Age  Gender   Department Job_Title  \
9120         9121  Margaret Daniels   37  Female  Engineering  Engineer   
4493         4494    Virginia Weber   46  Female    Marketing   Manager   

      Experience_Years Education_Level Location  Salary  
9120                14          Master   Austin  105000  
4493                23          Master  Chicago  150000  


In [7]:
data_frame.drop(columns=["Employee_ID","Name"],inplace=True)

In [8]:
print(data_frame.dtypes)

Age                  int64
Gender              object
Department          object
Job_Title           object
Experience_Years     int64
Education_Level     object
Location            object
Salary               int64
dtype: object


In [9]:
df_objects=[]
df_numbers=[]
for i in data_frame.columns:
        if(data_frame[i].dtype=="object"):
                df_objects.append(i)
        else:
                df_numbers.append(i)
print(df_objects)

['Gender', 'Department', 'Job_Title', 'Education_Level', 'Location']


In [10]:
for i in df_objects:
        print(data_frame[i].value_counts().shape)

(2,)
(6,)
(5,)
(3,)
(5,)


In [11]:
from sklearn.preprocessing import LabelEncoder
dict_of_encoders={}
for i in df_objects:
        encoder=LabelEncoder()
        data_frame[i]=encoder.fit_transform(data_frame[i])
        dict_of_encoders[encoder]=encoder.classes_


In [12]:
data_frame.dtypes

Age                 int64
Gender              int32
Department          int32
Job_Title           int32
Experience_Years    int64
Education_Level     int32
Location            int32
Salary              int64
dtype: object

In [13]:
print(dict_of_encoders)


{LabelEncoder(): array(['Female', 'Male'], dtype=object), LabelEncoder(): array(['Engineering', 'Finance', 'HR', 'Marketing', 'Product', 'Sales'],
      dtype=object), LabelEncoder(): array(['Analyst', 'Engineer', 'Executive', 'Intern', 'Manager'],
      dtype=object), LabelEncoder(): array(['Bachelor', 'Master', 'PhD'], dtype=object), LabelEncoder(): array(['Austin', 'Chicago', 'New York', 'San Francisco', 'Seattle'],
      dtype=object)}


In [14]:
dict_list=[i for i in dict_of_encoders.keys()]
for IDX,i in enumerate(df_objects):
        
        data_frame[i]=dict_list[IDX].inverse_transform(data_frame[i])

        

In [15]:
print(data_frame.dtypes)

Age                  int64
Gender              object
Department          object
Job_Title           object
Experience_Years     int64
Education_Level     object
Location            object
Salary               int64
dtype: object


In [16]:
for IDX,i in enumerate(df_objects):
        
        data_frame[i]=dict_list[IDX].fit_transform(data_frame[i])

        

In [17]:
print(data_frame.dtypes)

Age                 int64
Gender              int32
Department          int32
Job_Title           int32
Experience_Years    int64
Education_Level     int32
Location            int32
Salary              int64
dtype: object


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import HistGradientBoostingClassifier,RandomForestClassifier
from sklearn.svm import SVC
X=data_frame.drop(columns=["Salary"])
y=data_frame["Salary"]
x_train,x_test,y_train,y_test=train_test_split(X,y,random_state=42,train_size=0.7)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(7000, 7)
(7000,)
(3000, 7)
(3000,)


In [19]:
SVM=SVC()
hgbc=HistGradientBoostingClassifier()
rfc=RandomForestClassifier()
param_grid_svm={
        "C":[0.1,1,0.2,0.5],
        "kernel":['linear','rbf']
}
param_grid_hgbc={
        "learning_rate":[0.01,0.1,0.05,0.007],
        "max_iter":[100,200]

}
param_grid_rfc={
        "max_depth":[None,10,50],
        "n_estimators":[50,100]
}

In [42]:
grid_search_svm=GridSearchCV(estimator=SVM,param_grid=param_grid_svm,cv=5,scoring="accuracy",verbose=1)
grid_search_hgbc=GridSearchCV(estimator=hgbc,param_grid=param_grid_hgbc,cv=5,scoring="accuracy",verbose=1)
grid_search_rfc=GridSearchCV(estimator=rfc,param_grid=param_grid_rfc,cv=5,scoring="accuracy",verbose=1)
grid_search_svm.fit(x_train,y_train)
grid_search_rfc.fit(x_train,y_train)
grid_search_hgbc.fit(x_train,y_train)


Fitting 5 folds for each of 8 candidates, totalling 40 fits


C:\Users\yenug\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Users\yenug\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Fitting 5 folds for each of 8 candidates, totalling 40 fits


C:\Users\yenug\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5, estimator=HistGradientBoostingClassifier(),
             param_grid={'learning_rate': [0.01, 0.1, 0.05, 0.007],
                         'max_iter': [100, 200]},
             scoring='accuracy', verbose=1)

In [43]:
hgbc_1=grid_search_hgbc.best_estimator_
rfc_1=grid_search_rfc.best_estimator_
svm_1=grid_search_svm.best_estimator_
print(grid_search_svm.best_score_,grid_search_svm.best_params_)
print(grid_search_rfc.best_score_,grid_search_rfc.best_params_)
print(grid_search_hgbc.best_score_,grid_search_hgbc.best_params_)

0.42314285714285715 {'C': 0.5, 'kernel': 'linear'}
0.39728571428571435 {'max_depth': 10, 'n_estimators': 100}
0.3968571428571429 {'learning_rate': 0.007, 'max_iter': 100}


In [44]:
y_prediction1=hgbc_1.predict(x_test)
y_prediction2=rfc_1.predict(x_test)
y_prediction3=svm_1.predict(x_test)
print(accuracy_score(y_prediction1,y_test))
print(accuracy_score(y_prediction2,y_test))
print(accuracy_score(y_prediction3,y_test))

0.4076666666666667
0.38966666666666666
0.43


In [45]:
acc_df=pd.DataFrame(
        [[
                "SVM",accuracy_score(y_prediction3,y_test)*100
        ],
        [
                "rfc",accuracy_score(y_prediction2,y_test)*100
        ],
        [
                "hgbc",accuracy_score(y_prediction1,y_test)*100
        ]],
        columns=["Model_Type","Accuracy"]
)

In [46]:
print(acc_df)

  Model_Type   Accuracy
0        SVM  43.000000
1        rfc  38.966667
2       hgbc  40.766667


In [20]:
x_train_np=x_train.to_numpy()
x_test_np=x_test.to_numpy()
y_train_np=y_train.to_numpy()
y_test_np=y_test.to_numpy()
print(type(x_train),type(x_test),type(y_train),type(y_test))
print(type(x_train_np),type(x_test_np),type(y_train_np),type(y_test_np))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [21]:
import torch as t
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
from torchinfo import summary


In [22]:
x_train_tensor=t.from_numpy(x_train_np)
x_test_tensor=t.from_numpy(x_test_np)
y_test_tensor=t.from_numpy(y_test_np)
y_train_tensor=t.from_numpy(y_train_np)
print(type(x_train_tensor))

<class 'torch.Tensor'>


In [23]:
print(x_train_tensor.shape)
print(x_test_tensor.shape)
print(y_test_tensor.shape)
print(y_train_tensor.shape)


torch.Size([7000, 7])
torch.Size([3000, 7])
torch.Size([3000])
torch.Size([7000])


In [24]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train_tensor=scaler.fit_transform(x_train_tensor)
x_test_tensor=scaler.fit_transform(x_test_tensor)

In [35]:
class Datasetter(Dataset):
        def __init__(self,x,y):
                self.x=x
                self.y=y
        def __len__(self):
                return len(self.y)
        def __getitem__(self, index):
                return self.x[index],self.y[index]

In [36]:
train_dataset=Datasetter(x_train_tensor,y_train_tensor)
test_dataset=Datasetter(x_test_tensor,y_test_tensor)

In [38]:
print(len(train_dataset))
print(len(test_dataset))

7000
3000


In [40]:
train_dataloader=DataLoader(dataset=train_dataset,shuffle=True,batch_size=32,drop_last=True)
test_dataloader=DataLoader(dataset=test_dataset,shuffle=True,batch_size=32,drop_last=True)

In [42]:
print(len(train_dataloader))
print(len(test_dataloader))

218
93


In [ ]:
class Torch_model(nn.Module):
        def __init__(self,inp,hid,out):
                super().__init__()
                self.layer1=nn.Sequential(
                        nn.Linear(in_features=inp,out_features=hid),
                        nn.LeakyReLU(0.2,inplace=True)
                )
                self.layer2=nn.Sequential(
                        nn.Linear(in_features=hid,out_features=hid*2),
                        nn.LeakyReLU(0.2,inplace=True)
                )
                self.layer3=nn.Sequential(
                        nn.Linear(in_features=hid*2,out_features=hid),
                        nn.LeakyReLU(0.2,inplace=True)
                )
                self.regressor=nn.Sequential(
                        nn.Linear(hid,out)
                )
        def forward(self,x):
                return self.regressor(self.layer3(self.layer2(self.layer1(x))))


In [45]:
Model_torc=Torch_model(
        inp=7,
        hid=64,
        out=1
)
summary(Model_torc,input_size=(32,7))

Layer (type:depth-idx)                   Output Shape              Param #
Torch_model                              [32, 1]                   --
├─Sequential: 1-1                        [32, 64]                  --
│    └─Linear: 2-1                       [32, 64]                  512
│    └─LeakyReLU: 2-2                    [32, 64]                  --
├─Sequential: 1-2                        [32, 128]                 --
│    └─Linear: 2-3                       [32, 128]                 8,320
│    └─LeakyReLU: 2-4                    [32, 128]                 --
├─Sequential: 1-3                        [32, 64]                  --
│    └─Linear: 2-5                       [32, 64]                  8,256
│    └─LeakyReLU: 2-6                    [32, 64]                  --
├─Sequential: 1-4                        [32, 1]                   --
│    └─Linear: 2-7                       [32, 1]                   65
Total params: 17,153
Trainable params: 17,153
Non-trainable params: 0
Total mu

In [51]:
optimizer=optim.Adam(params=Model_torc.parameters(),lr=0.0001)
criterion=t.nn.HuberLoss(delta=1)

In [52]:
epochs=20
Model_torc.to(t.float32)
for epoch in range(epochs):
        train_loss=0
        test_loss=0
        Model_torc.train()
        for batch in train_dataloader:
                (x,y)=batch
                x=x.to(t.float32)
                y=y.to(t.float32)
                y_preds=Model_torc(x)
                loss=criterion(y_preds,y)
                train_loss+=loss.item()*32
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        Model_torc.eval()
        with t.inference_mode():
                for batch in test_dataloader:
                        (x,y)=batch
                        x=x.to(t.float32)
                        y=y.to(t.float32)
                        y_preds=Model_torc(x)
                        loss=criterion(y_preds,y)
                        test_loss+=loss.item()*32
        print(f"\033[1;36m Epoch : {epoch}\033[0m")
        print(f"\033[1;31m Train loss : {train_loss/len(train_dataloader.dataset)}\033[0m")
        print(f"\033[1;32m Test loss: {test_loss/len(test_dataloader.dataset)}\033[0m")
        print(f"\033[1;34m {"==="*60} \033[0m")



                

 Epoch : 0
 Train loss : 39173.819410714284
 Test loss: 39452.81210416667
 Epoch : 1
 Train loss : 39165.41091071429
 Test loss: 39384.9141875
 Epoch : 2
 Train loss : 39160.286375
 Test loss: 39418.64391666667
 Epoch : 3
 Train loss : 39163.75002678572
 Test loss: 39431.574145833336
 Epoch : 4
 Train loss : 39125.12996428571
 Test loss: 39341.08266666667
 Epoch : 5
 Train loss : 39148.722571428574
 Test loss: 39425.3759375
 Epoch : 6
 Train loss : 39181.40311607143
 Test loss: 39368.0824375
 Epoch : 7
 Train loss : 39156.27938392857
 Test loss: 39461.891229166664
 Epoch : 8
 Train loss : 39148.13166071429
 Test loss: 39365.15679166667
 Epoch : 9
 Train loss : 39163.69189285714
 Test loss: 39359.78927083333
 Epoch : 10
 Train loss : 39133.93579464286
 Test loss: 39384.000166666665
 Epoch : 11
 Train loss : 39149.363973214284
 Test loss: 39345.88247916667
 Epoch : 12
 Train loss : 39117.879
 Test loss: 39348.355395833336
 Epoch : 13
 Train loss : 39150.84094642857
 Test loss: 39403.2824

In [53]:
import tensorflow as tf
x_test_np=scaler.fit_transform(x_test_np)
x_train_np=scaler.fit_transform(x_train_np)


In [56]:
x_train_tf=tf.Variable(x_train_np)
x_test_tf=tf.Variable(x_test_np)
y_test_tf=tf.Variable(y_test_np)
y_train_tf=tf.Variable(y_train_np)

In [57]:
print(x_train_tf.shape)

(7000, 7)


In [59]:
train_dataset_tf=tf.data.Dataset.from_tensor_slices((x_train_tf,y_train_tf))
test_dataset_tf=tf.data.Dataset.from_tensor_slices((x_test_tf,y_test_tf))
train_dataset_tf=train_dataset_tf.shuffle(buffer_size=train_dataset_tf.cardinality().numpy())
test_dataset_tf=test_dataset_tf.shuffle(buffer_size=test_dataset_tf.cardinality().numpy())
train_dataloader_tf=train_dataset_tf.batch(32)
test_dataloader_tf=test_dataset_tf.batch(32)
train_dataloader_tf=train_dataloader_tf.prefetch(tf.data.AUTOTUNE)
test_dataloader_tf=test_dataloader_tf.prefetch(tf.data.AUTOTUNE)

In [60]:
print(len(train_dataloader_tf))
print(len(test_dataloader_tf))

219
94


In [65]:
Model_tf=tf.keras.Sequential(
        [       tf.keras.Input(shape=(7,)),
                tf.keras.layers.Dense(64,activation="relu"),
                tf.keras.layers.Dense(128,activation="relu"),
                tf.keras.layers.Dense(128,activation="relu"),
                tf.keras.layers.Dense(64,activation="relu"),
                tf.keras.layers.Dense(1)
        ]
)

In [66]:
Model_tf.compile(
        loss=tf.keras.losses.Huber(1),
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        metrics=["mae"]
)


In [67]:
Model_tf.fit(train_dataloader_tf,epochs=20,verbose=1)

Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 105409.9297 - mae: 105410.4219
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 20398.5059 - mae: 20399.0059
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 14975.7666 - mae: 14976.2666
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 13315.2158 - mae: 13315.7158
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 12409.6982 - mae: 12410.1982
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 11536.0234 - mae: 11536.5234
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 11051.2314 - mae: 11051.7314
Epoch 8/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 10388.5557 - mae: 10389.0557
Epoch 9/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 10094.9033 - mae: 10095.4033
Epoch 10/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 9713.9092 - mae: 9714.4092
Epoch 11/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 9112.0996 - mae: 9112.5996
Epoch 12/20
219/219 ━

In [68]:
Model_tf.evaluate(test_dataloader_tf)

94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 6062.6626 - mae: 6063.1621


[5896.89208984375, 5897.39208984375]

In [69]:
#tensorflow performed the best